<a href="https://colab.research.google.com/github/HeroBenHero/HeroColabs/blob/main/YTS_Api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
#@markdown **Enter The Movie Name**

Movie_Name = "Tom and Jerry" #@param {type:"string"}

def search_movies(query):
    base_url = "https://yts.mx/api/v2/list_movies.json"
    params = {"query_term": query}

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        # Check if the query was successful
        if data["status"] == "ok":
            movies = data["data"]["movies"]

            if movies:
                print("Search Results:")
                for idx, movie in enumerate(movies):
                    print(f"{idx + 1}. {movie['title']} ({movie['year']})")

                # Ask the user to pick a movie
                choice = int(input("Pick a movie (enter the number): ")) - 1

                if 0 <= choice < len(movies):
                    selected_movie = movies[choice]
                    return selected_movie
                else:
                    print("Invalid choice.")
                    return None

            else:
                print("No movies found.")
                return None

        else:
            print("Error:", data["status_message"])
            return None

    except requests.exceptions.RequestException as e:
        print("Error:", str(e))
        return None

def construct_magnet_url(selected_torrent, movie_title):
    if selected_torrent:
        magnet_url = f"magnet:?xt=urn:btih:{selected_torrent['hash']}&dn={movie_title}&tr=http://track.one:1234/announce"

        # Add recommended trackers
        recommended_trackers = [
            "udp://glotorrents.pw:6969/announce",
            "udp://tracker.opentrackr.org:1337/announce",
            "udp://torrent.gresille.org:80/announce",
            "udp://tracker.openbittorrent.com:80",
            "udp://tracker.coppersurfer.tk:6969",
            "udp://tracker.leechers-paradise.org:6969",
            "udp://p4p.arenabg.ch:1337",
            "udp://tracker.internetwarriors.net:1337"
        ]

        for tracker in recommended_trackers:
            magnet_url += f"&tr={tracker}"

        return magnet_url

def list_quality_and_size(selected_movie):
    if selected_movie:
        print(f"\nAvailable Qualities and Sizes for '{selected_movie['title']}' ({selected_movie['year']}):")
        torrents = selected_movie["torrents"]

        for idx, torrent in enumerate(torrents):
            print(f"{idx + 1}. Quality: {torrent['quality']}, Size: {torrent['size']}")

        # Ask the user to pick a quality
        choice = int(input("Pick a quality (enter the number): ")) - 1

        if 0 <= choice < len(torrents):
            selected_torrent = torrents[choice]
            return selected_torrent
        else:
            print("Invalid choice.")
            return None

def main():
    movie_name = Movie_Name
    selected_movie = search_movies(movie_name)

    if selected_movie:
        selected_torrent = list_quality_and_size(selected_movie)

        if selected_torrent:
            magnet_url = construct_magnet_url(selected_torrent, selected_movie['title'])
            print(f"\nMagnet URL for '{selected_movie['title']}' ({selected_movie['year']}):")
            print(magnet_url)
            print(f"\nTorrent URL for '{selected_movie['title']}' ({selected_movie['year']}):")
            print(selected_torrent["url"])

if __name__ == "__main__":
    main()


Search Results:
1. Tom and Jerry: The Fast and the Furry (2005)
2. Tom and Jerry: Robin Hood and His Merry Mouse (2012)
3. Tom and Jerry: Snowman's Land (2022)
4. Tom and Jerry: Santa's Little Helpers (2014)
5. Tom and Jerry: Willy Wonka and the Chocolate Factory (2017)
6. Tom and Jerry: A Nutcracker Tale (2007)
7. Tom and Jerry: Cowboy Up! (2021)
8. Jerry and Tom (1998)
9. Tom and Jerry Meet Sherlock Holmes (2010)
10. Tom and Jerry's Giant Adventure (2013)
11. Tom and Jerry: Back to Oz (2016)
12. Tom and Jerry: The Magic Ring (2001)
13. Tom and Jerry: The Movie (1992)
14. Tom and Jerry: The Lost Dragon (2014)
15. Tom and Jerry: Spy Quest (2015)
16. Tom and Jerry & The Wizard of Oz (2011)
17. Tom and Jerry Blast Off to Mars! (2005)
18. Tom and Jerry in Shiver Me Whiskers (2006)
Pick a movie (enter the number): 1

Available Qualities and Sizes for 'Tom and Jerry: The Fast and the Furry' (2005):
1. Quality: 720p, Size: 686.92 MB
2. Quality: 1080p, Size: 1.38 GB
Pick a quality (enter the 

In [ ]:
import requests
import re
#@markdown **Enter The Movie Name**

Movie_Name = "Tom and Jerry" #@param {type:"string"}

def search_movies(query):
    base_url = "https://yts.mx/api/v2/list_movies.json"
    params = {"query_term": query}

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        # Check if the query was successful
        if data["status"] == "ok":
            movies = data["data"]["movies"]

            if movies:
                return movies
            else:
                print("No movies found.")
                return None

        else:
            print("Error:", data["status_message"])
            return None

    except requests.exceptions.RequestException as e:
        print("Error:", str(e))
        return None

def get_highest_quality_torrent(movie):
    if "torrents" in movie:
        torrents = movie["torrents"]

        # Sort torrents by quality and size
        sorted_torrents = sorted(torrents, key=lambda x: (x["quality"], -parse_size(x["size"])), reverse=True)

        # Get the highest quality torrent
        highest_quality_torrent = sorted_torrents[-1]

        return highest_quality_torrent

def parse_size(size_str):
    # Regular expression to match sizes like "1.02 GB" or "800.6 MB"
    size_pattern = r'(\d+\.\d+)\s*(GB|MB)'
    match = re.match(size_pattern, size_str)

    if match:
        size, unit = match.groups()
        size = float(size)

        if unit == 'GB':
            return size * 1024  # Convert GB to MB
        elif unit == 'MB':
            return size

    return 0

def construct_magnet_url(movie, torrent):
    if movie and torrent:
        magnet_url = f"magnet:?xt=urn:btih:{torrent['hash']}&dn={movie['title'].replace(' ','+')}&tr=http://track.one:1234/announce"

        # Add recommended trackers
        recommended_trackers = [
            "udp://glotorrents.pw:6969/announce",
            "udp://tracker.opentrackr.org:1337/announce",
            "udp://torrent.gresille.org:80/announce",
            "udp://tracker.openbittorrent.com:80",
            "udp://tracker.coppersurfer.tk:6969",
            "udp://tracker.leechers-paradise.org:6969",
            "udp://p4p.arenabg.ch:1337",
            "udp://tracker.internetwarriors.net:1337"
        ]

        for tracker in recommended_trackers:
            magnet_url += f"&tr={tracker}"

        return magnet_url
def main():
    movie_name = Movie_Name
    movies = search_movies(movie_name)

    if movies:
        for idx, movie in enumerate(movies):
            highest_quality_torrent = get_highest_quality_torrent(movie)
            magnet_url = construct_magnet_url(movie, highest_quality_torrent)

            if magnet_url:
                print(f"Movie {idx + 1}:")
                print(f"Title: {movie['title']} ({movie['year']})")
                print(f"Highest Quality: {highest_quality_torrent['quality']}")
                print(f"Magnet URL: {magnet_url}\n")
            else:
                print(f"Failed to construct Magnet URL for {movie['title']} ({movie['year']}).")

if __name__ == "__main__":
    main()


Movie 1:
Title: Tom and Jerry: The Fast and the Furry (2005)
Highest Quality: 1080p
Magnet URL: magnet:?xt=urn:btih:7E259B19B505237C46D5FB66AE85E73216DE9FDB&dn=Tom+and+Jerry:+The+Fast+and+the+Furry&tr=http://track.one:1234/announce&tr=udp://glotorrents.pw:6969/announce&tr=udp://tracker.opentrackr.org:1337/announce&tr=udp://torrent.gresille.org:80/announce&tr=udp://tracker.openbittorrent.com:80&tr=udp://tracker.coppersurfer.tk:6969&tr=udp://tracker.leechers-paradise.org:6969&tr=udp://p4p.arenabg.ch:1337&tr=udp://tracker.internetwarriors.net:1337

Movie 2:
Title: Tom and Jerry: Robin Hood and His Merry Mouse (2012)
Highest Quality: 1080p
Magnet URL: magnet:?xt=urn:btih:90C3ED15D786A309700C6B086DD3283A8F09F137&dn=Tom+and+Jerry:+Robin+Hood+and+His+Merry+Mouse&tr=http://track.one:1234/announce&tr=udp://glotorrents.pw:6969/announce&tr=udp://tracker.opentrackr.org:1337/announce&tr=udp://torrent.gresille.org:80/announce&tr=udp://tracker.openbittorrent.com:80&tr=udp://tracker.coppersurfer.tk:69